### Configuration

In [0]:
### google drive mount ###
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
### configuration ###
# import numpy and pandas
import numpy as np
import pandas as pd

# used for dates
import datetime as dt

# for collecting open api data
import urllib.request
import json
from pandas.io.json import json_normalize

# config
DATA_PATH = "/content/gdrive/My Drive/data/" 
# local 실행시
# DATA_PATH = "../data/"  #코드파일의 위치는 ~/code/ data폴더의 위치는 ~/code/data/
ENCODING = 'CP949'

In [0]:
### collect & concat data function ###

def make_path(add_path):
    path_list = [DATA_PATH]
    path_list.append(add_path)
    return ''.join(path_list)

def read_file(path, encode=ENCODING):
    df = pd.read_csv(path, encoding=encode, dtype=str)
    return df

def read_files(path, encode=ENCODING):
    df_from_each_file = (
        pd.read_csv(f, encoding=encode, dtype=str) for f in path
        )
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    return concatenated_df

### 파생변수 - 1) 시간대별 차량수

In [0]:
# 수집된 시간표 데이터 프레임으로 가져오기
tc = pd.read_csv(make_path("timechart.csv"),
                 dtype=str,
                 usecols=['STATION_CD','ARRIVETIME'])
tc

,STATION_CD,ARRIVETIME
0,0150,07:17:00
1,0150,23:58:30
2,0150,08:06:00
3,0150,05:19:30
4,0150,00:00:00
...,...,...
84620,0430,23:14:30
84621,0430,23:22:30
84622,0430,23:31:30
84623,0430,23:41:30


In [0]:
# 시간대 추출
tc['HOUR']=tc['ARRIVETIME'].str.split(':').str[0]
tc.head(1)

,STATION_CD,ARRIVETIME,HOUR
0,0150,07:17:00,07


In [0]:
# 시간대 변환 : 06시보다 작으면 BEFORE_06, 24이면 AFTER_24
tc['HOUR'] = tc['HOUR'].apply(lambda x: 
                              'B' if ( int(x) < 6 and int(x) != 0 )
                              else 'A' if ( int(x) == 0 or int(x) == 24 )
                              else str(x))
tc.HOUR.unique()

array(['07', '23', '08', 'B', 'A', '06', '09', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22'], dtype=object)

In [0]:
tc_f = tc.loc[(tc.HOUR != 'B') & (tc.HOUR != 'A')]
tc_f.HOUR.unique()

array(['07', '23', '08', '06', '09', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22'], dtype=object)

In [0]:
# 역별/시간대별 차량수 
tc = tc_f
tc_cnt = tc.groupby(["STATION_CD", "HOUR"], as_index=False)["ARRIVETIME"].count()
tc_cnt.rename(columns={'ARRIVETIME' : 'TRAIN_CNT'}, inplace=True)
tc_cnt

,STATION_CD,HOUR,TRAIN_CNT
0,0150,06,27
1,0150,07,31
2,0150,08,34
3,0150,09,32
4,0150,10,27
...,...,...,...
4331,1958,19,7
4332,1958,20,7
4333,1958,21,5
4334,1958,22,6


In [0]:
# 테이블 정의서대로 컬럼 정리
tc_cnt.columns = ['STA_CD', 'BAS_TIME', 'TRAIN_CNT']
tc_cnt

,STA_CD,BAS_TIME,TRAIN_CNT
0,0150,06,27
1,0150,07,31
2,0150,08,34
3,0150,09,32
4,0150,10,27
...,...,...,...
4331,1958,19,7
4332,1958,20,7
4333,1958,21,5
4334,1958,22,6


In [0]:
tc_cnt.isnull().sum()

STA_CD       0
BAS_TIME     0
TRAIN_CNT    0
dtype: int64

In [0]:
tc_cnt['STA_CD'] = tc_cnt['STA_CD'].str.strip()
tc_cnt

,STA_CD,BAS_TIME,TRAIN_CNT
0,0150,06,27
1,0150,07,31
2,0150,08,34
3,0150,09,32
4,0150,10,27
...,...,...,...
4331,1958,19,7
4332,1958,20,7
4333,1958,21,5
4334,1958,22,6


### 파생변수 - 2) 시간대별 혼잡도
**혼잡도 = 상하행 승하차인원 / 54명 *10량 * 차량수**

In [0]:
# 승하차인원 가져오기
df = read_file(make_path("유동인구.csv"))
df['INOUT_CNT'] = pd.to_numeric(df['승하차인원'])
df['FLT_CNT'] = pd.to_numeric(df['유동인구수'])
df

,날짜,호선,역번호,역명,시간대,date,year,month,day,구,승하차인원,일자,유동인구수,TRANSFER,INOUT_CNT,FLT_CNT
0,2019-10-01,1,0150,서울역,06,2019-10-01,2019,10,1,중구,2446,20191001,153060.0,1,2446,153060.0
1,2019-10-01,1,0150,서울역,07,2019-10-01,2019,10,1,중구,6448,20191001,231170.0,1,6448,231170.0
2,2019-10-01,1,0150,서울역,08,2019-10-01,2019,10,1,중구,13578,20191001,370810.0,1,13578,370810.0
3,2019-10-01,1,0150,서울역,09,2019-10-01,2019,10,1,중구,9203,20191001,428280.0,1,9203,428280.0
4,2019-10-01,1,0150,서울역,10,2019-10-01,2019,10,1,중구,4982,20191001,453590.0,1,4982,453590.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53095,2019-10-31,4,0434,남태령,19,2019-10-31,2019,10,31,서초구,147,20191031,464040.0,0,147,464040.0
53096,2019-10-31,4,0434,남태령,20,2019-10-31,2019,10,31,서초구,110,20191031,433020.0,0,110,433020.0
53097,2019-10-31,4,0434,남태령,21,2019-10-31,2019,10,31,서초구,85,20191031,409520.0,0,85,409520.0
53098,2019-10-31,4,0434,남태령,22,2019-10-31,2019,10,31,서초구,70,20191031,385540.0,0,70,385540.0


In [0]:
# 승하차인원 컬럼 이름 테이블 정의서에 맞춰 바꾸기 
subset= df[['날짜', '호선', '역번호', '역명', '시간대', 'INOUT_CNT', 'FLT_CNT', 'TRANSFER']]
subset.columns = ['BAS_DT', 'LINE_NO', 'STA_CD', 'STA_NM', 'BAS_TIME', 'INOUT_CNT', 'FLT_CNT', 'TRANSFER']
subset['STA_CD'] = subset['STA_CD'].str.strip()
subset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,BAS_DT,LINE_NO,STA_CD,STA_NM,BAS_TIME,INOUT_CNT,FLT_CNT,TRANSFER
0,2019-10-01,1,0150,서울역,06,2446,153060.0,1
1,2019-10-01,1,0150,서울역,07,6448,231170.0,1
2,2019-10-01,1,0150,서울역,08,13578,370810.0,1
3,2019-10-01,1,0150,서울역,09,9203,428280.0,1
4,2019-10-01,1,0150,서울역,10,4982,453590.0,1


In [0]:
subset = subset.loc[(subset.BAS_TIME != 'BEFORE_06') & (subset.BAS_TIME != 'AFTER_24')]
subset.BAS_TIME.unique()

array(['06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'], dtype=object)

In [0]:
# 상하행 합치기 
onoff =subset.groupby(["BAS_DT", "STA_CD", "STA_NM", "BAS_TIME", "TRANSFER"], as_index=False)["INOUT_CNT"].sum()
onoff

,BAS_DT,STA_CD,STA_NM,BAS_TIME,TRANSFER,INOUT_CNT
0,2019-10-01,0150,서울역,06,1,2446
1,2019-10-01,0150,서울역,07,1,6448
2,2019-10-01,0150,서울역,08,1,13578
3,2019-10-01,0150,서울역,09,1,9203
4,2019-10-01,0150,서울역,10,1,4982
...,...,...,...,...,...,...
53095,2019-10-31,0434,남태령,19,0,147
53096,2019-10-31,0434,남태령,20,0,110
53097,2019-10-31,0434,남태령,21,0,85
53098,2019-10-31,0434,남태령,22,0,70


In [0]:
# 차량내평균인원 가져오기 
df = read_file(make_path("차량내혼잡도_2017.csv"))
df.columns = ['idx', 'gubun', 'LINE_NO', 'STA_NM', 'STA_CD', 'INOUT_CD','BAS_TIME','AVGCNT']
df['AVGCNT'] = pd.to_numeric(df['AVGCNT'])
df['STA_CD'] = df['STA_CD'].str.zfill(4).str.strip()
df

,idx,gubun,LINE_NO,STA_NM,STA_CD,INOUT_CD,BAS_TIME,AVGCNT
0,0,평일,1,동대문,0155,1,05,136.0
1,1,평일,1,동대문,0155,1,06,161.6
2,2,평일,1,동대문,0155,1,07,262.4
3,3,평일,1,동대문,0155,1,08,460.0
4,4,평일,1,동대문,0155,1,09,416.8
...,...,...,...,...,...,...,...,...
4515,4515,평일,4,회현,0425,2,20,578.4
4516,4516,평일,4,회현,0425,2,21,667.2
4517,4517,평일,4,회현,0425,2,22,784.0
4518,4518,평일,4,회현,0425,2,23,528.8


In [0]:
# 차량내평균인원 상하행 합치기 
avgcnt =df.groupby(["STA_CD","BAS_TIME"], as_index=False)["AVGCNT"].sum()
avgcnt

,STA_CD,BAS_TIME,AVGCNT
0,0150,05,305.6
1,0150,06,562.4
2,0150,07,1098.4
3,0150,08,1828.0
4,0150,09,1240.0
...,...,...,...
2255,0434,20,1627.2
2256,0434,21,1394.4
2257,0434,22,1556.8
2258,0434,23,1382.4


In [0]:
# 차량내평균인원 시간대, CD 변환
avgcnt['BAS_TIME'] = avgcnt['BAS_TIME'].apply(lambda x: 
                                            'BEFORE_06' if int(x) == 5
                                            else 'AFTER_24' if int(x) == 24
                                            else x)
avgcnt

,STA_CD,BAS_TIME,AVGCNT
0,0150,BEFORE_06,305.6
1,0150,06,562.4
2,0150,07,1098.4
3,0150,08,1828.0
4,0150,09,1240.0
...,...,...,...
2255,0434,20,1627.2
2256,0434,21,1394.4
2257,0434,22,1556.8
2258,0434,23,1382.4


In [0]:
avgcnt = avgcnt.loc[(avgcnt.BAS_TIME != 'BEFORE_06') & (avgcnt.BAS_TIME != 'AFTER_24')]
avgcnt.BAS_TIME.unique()

array(['06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'], dtype=object)

In [0]:
avgcnt.isnull().sum()

STA_CD      0
BAS_TIME    0
AVGCNT      0
dtype: int64

In [0]:
# 차량수 붙이기
merge1 = pd.merge(onoff, tc_cnt, how='left', on=['STA_CD', 'BAS_TIME'])
merge1

,BAS_DT,STA_CD,STA_NM,BAS_TIME,TRANSFER,INOUT_CNT,TRAIN_CNT
0,2019-10-01,0150,서울역,06,1,2446,27
1,2019-10-01,0150,서울역,07,1,6448,31
2,2019-10-01,0150,서울역,08,1,13578,34
3,2019-10-01,0150,서울역,09,1,9203,32
4,2019-10-01,0150,서울역,10,1,4982,27
...,...,...,...,...,...,...,...
53095,2019-10-31,0434,남태령,19,0,147,17
53096,2019-10-31,0434,남태령,20,0,110,15
53097,2019-10-31,0434,남태령,21,0,85,16
53098,2019-10-31,0434,남태령,22,0,70,10


In [0]:
merge1['STA_CD'] = merge1['STA_CD'].str.strip()
avgcnt['STA_CD'] = avgcnt['STA_CD'].str.strip()
merge1['BAS_TIME'] = merge1['BAS_TIME'].str.strip()
avgcnt['BAS_TIME'] = avgcnt['BAS_TIME'].str.strip()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
# 차량내평균인원 붙이기
merge2 = pd.merge(merge1, avgcnt, how='left', on=['STA_CD', 'BAS_TIME'])
merge2

,BAS_DT,STA_CD,STA_NM,BAS_TIME,TRANSFER,INOUT_CNT,TRAIN_CNT,AVGCNT
0,2019-10-01,0150,서울역,06,1,2446,27,562.4
1,2019-10-01,0150,서울역,07,1,6448,31,1098.4
2,2019-10-01,0150,서울역,08,1,13578,34,1828.0
3,2019-10-01,0150,서울역,09,1,9203,32,1240.0
4,2019-10-01,0150,서울역,10,1,4982,27,890.4
...,...,...,...,...,...,...,...,...
53095,2019-10-31,0434,남태령,19,0,147,17,2172.8
53096,2019-10-31,0434,남태령,20,0,110,15,1627.2
53097,2019-10-31,0434,남태령,21,0,85,16,1394.4
53098,2019-10-31,0434,남태령,22,0,70,10,1556.8


In [0]:
merge2.isnull().sum()

BAS_DT          0
STA_CD          0
STA_NM          0
BAS_TIME        0
TRANSFER        0
INOUT_CNT       0
TRAIN_CNT       0
AVGCNT       3150
dtype: int64

In [0]:
# 결측치를 시간대 평균으로 fillna 
fill_mean_func = lambda g: g.fillna(g.mean())
merge2_fna = merge2.copy().groupby('BAS_TIME', as_index=False).apply(fill_mean_func)
merge2_fna

BAS_DT STA_CD       STA_NM  ... INOUT_CNT TRAIN_CNT  AVGCNT
0  0      2019-10-01   0150          서울역  ...      2446        27   562.4
   18     2019-10-01   0151           시청  ...      1006        28   470.4
   36     2019-10-01   0152           종각  ...      1589        28   427.2
   54     2019-10-01   0153         종로3가  ...       738        27   458.4
   72     2019-10-01   0154         종로5가  ...       824        28   466.4
...              ...    ...          ...  ...       ...       ...     ...
17 53027  2019-10-31   0430  이촌(국립중앙박물관)  ...       426        11   833.6
   53045  2019-10-31   0431      동작(현충원)  ...       140         9   931.2
   53063  2019-10-31   0432    총신대입구(이수)  ...      1217        10   935.2
   53081  2019-10-31   0433           사당  ...      1825         9   953.6
   53099  2019-10-31   0434          남태령  ...        38         7  1382.4

[53100 rows x 8 columns]

In [0]:
merge2_fna.isnull().sum()

BAS_DT       0
STA_CD       0
STA_NM       0
BAS_TIME     0
TRANSFER     0
INOUT_CNT    0
TRAIN_CNT    0
AVGCNT       0
dtype: int64

In [0]:
merge2_fna = merge2_fna.reset_index(drop=True)
merge2_fna

,BAS_DT,STA_CD,STA_NM,BAS_TIME,TRANSFER,INOUT_CNT,TRAIN_CNT,AVGCNT
0,2019-10-01,0150,서울역,06,1,2446,27,562.4
1,2019-10-01,0151,시청,06,1,1006,28,470.4
2,2019-10-01,0152,종각,06,0,1589,28,427.2
3,2019-10-01,0153,종로3가,06,1,738,27,458.4
4,2019-10-01,0154,종로5가,06,0,824,28,466.4
...,...,...,...,...,...,...,...,...
53095,2019-10-31,0430,이촌(국립중앙박물관),23,1,426,11,833.6
53096,2019-10-31,0431,동작(현충원),23,1,140,9,931.2
53097,2019-10-31,0432,총신대입구(이수),23,1,1217,10,935.2
53098,2019-10-31,0433,사당,23,1,1825,9,953.6


In [0]:
# 혼잡도 = 상하행 승하차인원 + 차량내평균인원 / 54명 *10량 * 상하행 차량수
onoff_f = merge2_fna
onoff_f['CURPPL'] = onoff_f['INOUT_CNT'] + onoff_f['AVGCNT']
onoff_f['FULL'] = onoff_f['TRAIN_CNT'].astype(int) * 54 * 10
onoff_f['CONGES_RATE'] = (onoff_f['CURPPL'] / onoff_f['FULL'])*100
onoff_f

,BAS_DT,STA_CD,STA_NM,BAS_TIME,TRANSFER,INOUT_CNT,TRAIN_CNT,AVGCNT,CURPPL,FULL,CONGES_RATE
0,2019-10-01,0150,서울역,06,1,2446,27,562.4,3008.4,14580,20.633745
1,2019-10-01,0151,시청,06,1,1006,28,470.4,1476.4,15120,9.764550
2,2019-10-01,0152,종각,06,0,1589,28,427.2,2016.2,15120,13.334656
3,2019-10-01,0153,종로3가,06,1,738,27,458.4,1196.4,14580,8.205761
4,2019-10-01,0154,종로5가,06,0,824,28,466.4,1290.4,15120,8.534392
...,...,...,...,...,...,...,...,...,...,...,...
53095,2019-10-31,0430,이촌(국립중앙박물관),23,1,426,11,833.6,1259.6,5940,21.205387
53096,2019-10-31,0431,동작(현충원),23,1,140,9,931.2,1071.2,4860,22.041152
53097,2019-10-31,0432,총신대입구(이수),23,1,1217,10,935.2,2152.2,5400,39.855556
53098,2019-10-31,0433,사당,23,1,1825,9,953.6,2778.6,4860,57.172840


In [0]:
onoff_f.dtypes

BAS_DT          object
STA_CD          object
STA_NM          object
BAS_TIME        object
TRANSFER        object
INOUT_CNT        int64
TRAIN_CNT        int64
AVGCNT         float64
CURPPL         float64
FULL             int64
CONGES_RATE    float64
dtype: object

In [0]:
f = onoff_f[(onoff_f.BAS_DT != '2019-10-06') & (onoff_f.BAS_DT != '2019-10-13') & (onoff_f.BAS_DT != '2019-10-20') & (onoff_f.BAS_DT != '2019-10-27') & (onoff_f.BAS_DT != '2019-10-03') & (onoff_f.BAS_DT != '2019-10-09')]
f.BAS_DT.unique()

array(['2019-10-01', '2019-10-02', '2019-10-04', '2019-10-05',
       '2019-10-07', '2019-10-08', '2019-10-10', '2019-10-11',
       '2019-10-12', '2019-10-14', '2019-10-15', '2019-10-16',
       '2019-10-17', '2019-10-18', '2019-10-19', '2019-10-21',
       '2019-10-22', '2019-10-23', '2019-10-24', '2019-10-25',
       '2019-10-26', '2019-10-28', '2019-10-29', '2019-10-30',
       '2019-10-31'], dtype=object)

In [0]:
congest_rate_path = make_path("congest_rate.csv")
onoff_f.to_csv(congest_rate_path, encoding='CP949', index = False)